In [8]:
using HerbGrammar, HerbData, HerbSearch, HerbInterpret, HerbBenchmarks

import HerbInterpret.interpret

The original benchmark uses a wide set of instructions, which are divided into subgroups. The instructions were originally written in Clojure and can be found [here](https://github.com/thelmuth/Clojush/tree/psb2-v1.0/src/clojush/instructions). 

We translated these to Julia functions using ChatGPT with the prompt: "Can you give me julia functions for the following clojure definitions:". Each of the instruction sets can be found in the `grammars` directory. We create a `cfgrammar` for each set, so they can be imported individually. 

In [9]:
include("grammars/grammar_char.jl")
include("grammars/grammar_gtm.jl")
include("grammars/grammar_bool.jl")
include("grammars/grammar_util.jl")
include("grammars/grammar_numbers.jl")

SystemError: SystemError: opening file "/Users/issahanou/Work/code/HerbBenchmarks/src/data/PSB2_2021/grammars/grammar_numbers.jl": No such file or directory

In [10]:
g_general = @cfgrammar begin
    State = keep_number_reasonable
end

g_char = @cfgrammar begin
    State = char_allfromstring | char_fromfloat | char_fromfloat | char_isletter | char_isdigit | char_iswhitespace | char_lowercase | char_uppercase
end

g_gtm = @cfgrammar begin
    State = init_gtm | ensure_instruction_map | load_track | dump_track | trace | gtm_left | gtm_right | gtm_inc_delay | gtm_dec_delay | gtm_dub1
end

g_bool = @cfgrammar begin
    State = boolean_and | boolean_or | boolean_xor | boolean_not | boolean_invert_first_then_and | boolean_invert_second_then_and | boolean_fromfloat | boolean_fromfloat
end

g_float = @cfgrammar begin
    State = integer_add | integer_sub | integer_mult | integer_div | integer_mod | integer_lt | integer_gt | integer_lte | integer_gte | integer_fromboolean | integer_fromfloat | integer_fromstring | integer_fromchar | integer_min | integer_max | integer_inc | integer_dec | integer_abs | integer_negate | integer_pow
end

g_float = @cfgrammar begin
    State = float_add | float_sub | float_mult | float_div | float_mod | float_lt | float_gt | float_lte | float_gte | float_fromboolean | float_fromfloat | float_fromstring | float_fromchar | float_min | float_max | float_inc | float_dec | float_abs | float_negate | float_pow | float_arctan | float_arccos | float_arcsin | float_floor | float_ceiling | float_log10 | float_log2 | float_square | float_sqrt | float_tan | float_cos | float_sin
end

g_string = @cfgrammar begin
    State = string_frominteger | string_fromfloat | string_fromchar | string_fromboolean | string_concat | string_conjchar | string_take | string_substring | string_first | string_last | string_nth | string_rest | string_butlast | string_length | string_reverse | string_parse_to_chars | string_split | string_emptystring | string_containschar | string_indexofchar | string_occurrencesofchar | string_replace | string_replacefirst | string_replacechar | string_replacefirstchar | string_removechar | string_setchar | string_capitalize | string_uppercase | string_lowercase | exec_string_iterate | string_sort | string_includes | string_indexof
end

g_random = @cfgrammar begin
    State = boolean_rand | float_rand| integer_rand | string_rand | code_rand | code_rand_atom | char_rand
end


1: State = boolean_rand
2: State = float_rand
3: State = integer_rand
4: State = string_rand
5: State = code_rand
6: State = code_rand_atom
7: State = char_rand


In [11]:
function interpret(tab::SymbolTable, ex::Expr)
    """
    Attempt at an interpreter, not complete.
    """
    args = ex.args
    input = tab[:input]
    index = tab[:gtm][:position]

    if ex.head == :call
        if ex.args[1] in keys(tab)
            if length(args) > 1
                println("next step")
                interpret(tab, ex.args[2])
            else
                println("execute")
                tab[ex.args[1]](tab)
            end
        end
        error("Expression type not supported: $(ex.head)")
    end
end

interpret (generic function with 5 methods)

This creates a small example of how to use the grammar. We can merge different different grammars together, each should be annotated by its name (e.g., `Dict{Symbol, Any}(:char => SymbolTable(g_char))`).

Then, we create an input dictionary (e.g., `Dict(:input => "a")`), turn it into a state using `init_gtm` and finally merge it with the grammar dictionaries. 

Finally, we create an example of a program and interpret it. 

In [12]:
tab_char = Dict{Symbol, Any}(:char => SymbolTable(g_char))
tab_gtm = SymbolTable(g_gtm)
tab = merge(tab_char, tab_gtm)
input = Dict(:input => "a")
input = init_gtm(input)
input = merge(tab, input)
ex = :(char_isletter()) # should be true

interpret(input, ex)

ErrorException: Expression type not supported: call